### 1. 경로설정

In [2]:
import os
PATH = 'C:/Users/user/DataPreprocessing/ModelingDataset/data'
os.chdir(PATH)

### 2. 모듈 임포트

In [3]:
import csv
import pandas as pd
import numpy as np

# def portmis()

In [1]:
def portmis():
    df_portmis = pd.read_csv("./portmis(221130-221230).csv", encoding = 'cp949')
    
    # 선박용도 컬럼: 국제카페리, 여객선, 기타선 삭제
    df_portmis.drop(df_portmis[(df_portmis['선박용도'] == '여객선') | (df_portmis['선박용도'] == '국제카페리') | (df_portmis['선박용도'] == '기타선') ].index, inplace=True)
    
    # 입출항 컬럼: 입항/출항 中 출항 삭제
    df_portmis = df_portmis.drop(index = df_portmis[df_portmis['입출'] == '출항'].index)
    
    # 구분 컬럼: 최종/최초 中 '최초' 삭제
    df_portmis = df_portmis.drop(index = df_portmis[df_portmis['구분'] == '최초'].index).reset_index(drop=True)

    # 입항횟수, 입항횟수.1 컬럼: 모두 삭제
    df_portmis = df_portmis.drop(['입항횟수', '입항횟수.1'], axis = 1)

    return df_portmis

In [5]:
portmis = portmis()

# def match_mmsi()
- 호출부호, mmsi 매칭

In [18]:
def match_mmsi():
    df_mmsi = pd.read_csv("./호출부호_mmsi매칭.csv", encoding = 'cp949', index_col=0)
    df_mmsi.reset_index(drop = True, inplace = True)

    # pd.isin 으로 df_portmis에서 df_mmsi에 있는 호출부호 행만 추출
    df_schedules = portmis[portmis['호출부호'].isin(list(df_mmsi['호출부호']))].reset_index(drop=True)

    # df_portmis, df_mmsi를 호출부호 컬럼으로 병합하기
    df_schedules = pd.merge(portmis, df_mmsi, on = '호출부호', how = 'inner', indicator = True)

    # df_schedules 컬럼 순서 변경(편의상)
    df_schedules = df_schedules[['항명', '호출부호', 'mmsi', '선명','구분','외내','입출','총톤수','입항일시','출항일시','전출항지','선박용도']]

    # 중복데이터 삭제
    df_schedules = df_schedules.drop_duplicates(['mmsi','입항일시'], keep = 'first').reset_index(drop = True)

    # 결측치 제거
    df_schedules.dropna(inplace = True)

    # 인덱스 정리
    df_schedules.reset_index(drop=True, inplace= True)

    # mmsi값 int형으로 변환
    df_schedules['mmsi'] = df_schedules['mmsi'].astype(int)
    return df_schedules

In [19]:
match_mmsi()

,항명,호출부호,mmsi,선명,구분,외내,입출,총톤수,입항일시,출항일시,전출항지,선박용도
0,인천,3FDX9,371369000,POS BANGKOK,최종,외항,입항,"18,085",2022-11-30 00:36,2022-11-30 13:54,SHANTOU,풀컨테이너선
1,인천,3FDX9,371369000,POS BANGKOK,최종,외항,입항,"18,085",2022-12-15 07:05,2022-12-15 21:20,SHANTOU,풀컨테이너선
2,인천,3FDX9,371369000,POS BANGKOK,최종,외항,입항,"18,085",2022-12-30 02:55,2022-12-30 17:10,SHEKOU PT,풀컨테이너선
3,인천,D7SK,441791000,코리아비너스,최종,내항,입항,"29,543",2022-11-30 08:55,2022-12-01 08:55,여수,석유제품 운반선
4,인천,D7SK,441791000,코리아비너스,최종,내항,입항,"29,543",2022-12-04 13:45,2022-12-05 13:50,여수,석유제품 운반선
...,...,...,...,...,...,...,...,...,...,...,...,...
878,인천,3E4021,352001275,DONG HUA 17,최종,외항,입항,"2,967",2022-12-30 09:00,2022-12-31 01:40,LAIZHOU,일반화물선
879,인천,VRUS7,477842900,SITC XINGDE,최종,외항,입항,"18,820",2022-12-30 13:00,2022-12-31 03:05,XIAMEN,풀컨테이너선
880,인천,TJM0036,613003641,DORIS,최종,외항,입항,"1,941",2022-12-30 18:10,2022-12-31 16:05,여수,일반화물선
881,인천,V2GK7,305977000,MCC SHENZHEN,최종,외항,입항,"17,769",2022-12-30 18:55,2022-12-31 07:15,SHANGHAI,풀컨테이너선


### 2. df_schedules 데이터 csv 저장

In [20]:
match_mmsi().to_csv("./schedules.csv", sep = ',',encoding = 'cp949')